# Build giant tables

In [1]:
import os
import numpy as np
import pandas as pd
import scipy.io
from scipy.stats import iqr
from scipy import interpolate

In [2]:
# Load HUP_implant_dates.xlsx
patients_df = pd.read_excel("../../Data/HUP_implant_dates.xlsx")
patients_df

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,225,HUP225_phaseII,2021-10-18,07:15:00,2021-10-26 17:30:00,58.5
1,224,HUP224_phaseII,2021-10-13,07:15:00,2021-10-20 00:00:00,85.5
2,223,HUP223_phaseII,2021-09-29,07:15:00,2021-10-08 08:21:00,101.4
3,221,HUP221_phaseII,2021-08-16,07:15:00,2021-08-23 00:00:00,124.3
4,219,HUP219_phaseII,2021-07-12,07:15:00,2021-07-16 08:18:00,101.6
...,...,...,...,...,...,...
75,141,HUP141_phaseII,2017-05-24,07:15:00,2017-06-01 00:00:00,85.7
76,140,HUP140_phaseII_D01-D02,2017-05-10,07:15:00,2017-05-19 00:00:00,56.7
77,139,HUP139_phaseII,2017-04-26,07:15:00,2017-05-09 00:00:00,69.8
78,138,HUP138_phaseII,2017-04-12,07:15:00,2017-04-20 00:00:00,84.4


In [3]:
# Create a mapping between patient ids and the index of the patient in the patients_df dataframe
patient_hup_id_to_index = {}
for i, patient_id in enumerate(patients_df["hup_id"]):
    patient_hup_id_to_index[patient_id] = i

In [4]:
def get_patient_hup_ids(directory):
    # List all files in the directory
    files = os.listdir(directory)

    # Filter out files based on the given pattern and extract patient_hup_id as integers
    patient_hup_ids = [
        int(f.split("_")[1].split(".")[0])
        for f in files
        if f.startswith("HUP_") and f.endswith(".npy")
    ]

    return patient_hup_ids


directory = "../../Data/synchrony/all/broadband_new_after_fill/"
completed_hup_ids = get_patient_hup_ids(directory)
completed_hup_ids.sort()
print(completed_hup_ids)
len(completed_hup_ids)

[139, 143, 146, 155, 157, 160, 161, 162, 163, 164, 165, 171, 177, 178, 182, 188, 192, 206, 210, 217]


20

In [5]:
# Only keep the rows in patients_df that correspond to the completed_hup_ids
patients_df = patients_df[patients_df["hup_id"].isin(completed_hup_ids)]
# reset the index of patients_df
patients_df = patients_df.reset_index(drop=True)
patients_df

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,217,HUP217_phaseII,2021-05-17,07:15:00,2021-06-04 00:00:00,58.5
1,210,HUP210_phaseII,2020-11-30,07:15:00,2020-12-11 00:00:00,66.6
2,206,HUP206_phaseII,2020-08-24,07:15:00,2020-09-09 00:00:00,97.3
3,192,HUP192_phaseII,2019-04-24,07:15:00,2019-05-03 15:04:00,49.4
4,188,HUP188_phaseII,2019-03-13,07:15:00,2019-03-20 00:00:00,70.7
5,182,HUP182_phaseII,2018-12-12,07:15:00,2018-12-24 00:00:00,55.8
6,178,HUP178_phaseII,2018-09-12,07:15:00,2018-09-21 00:00:00,79.4
7,177,HUP177_phaseII,2018-08-22,07:15:00,2018-08-31 00:00:00,88.4
8,171,HUP171_phaseII,2018-06-13,07:15:00,2018-06-27 00:00:00,107.2
9,165,HUP165_phaseII,2018-03-14,07:15:00,2018-04-04 00:00:00,95.3


In [6]:
ieeg_offset_df = pd.read_excel("../../Data/ieeg_offset_new.xlsx")
ieeg_offset_df

,hup_id,ieeg_offset_1,ieeg_offset_2,ieeg_offset_3,ieeg_offset_4
0,225,136590,NaN,NaN,NaN
1,224,135178,NaN,NaN,NaN
2,223,118454,NaN,NaN,NaN
3,221,135123,NaN,NaN,NaN
4,219,119758,NaN,NaN,NaN
...,...,...,...,...,...
75,141,133760,NaN,NaN,NaN
76,140,126678,283535.0,NaN,NaN
77,139,133613,NaN,NaN,NaN
78,138,134989,NaN,NaN,NaN


In [7]:
all_med_names = []

for i, row in patients_df.iterrows():
    # Get patient id and weight
    patient_hup_id = row.hup_id

    # Load HUP_{patient_hup_id}.npy from ../../Data/medications
    aed_np_file = np.load(
        f"../../Data/medications/HUP_{patient_hup_id}.npy", allow_pickle=True
    )

    all_dose_curves_plot = aed_np_file[0]
    all_tHr_plot = aed_np_file[1]
    all_med_names_plot = aed_np_file[2]

    # Plot dose curves
    for med_name in all_med_names_plot:
        all_med_names.append(med_name)

all_med_names = np.unique(np.array(all_med_names, dtype=str))
all_med_names

array(['brivaracetam', 'carbamazepine', 'clobazam', 'clonazepam',
       'eslicarbazepine', 'felbamate', 'gabapentin', 'lacosamide',
       'lamotrigine', 'levetiracetam', 'lorazepam', 'oxcarbazepine',
       'phenytoin', 'pregabalin', 'topiramate', 'zonisamide'],
      dtype='<U15')

In [8]:
# Load aed_ref_ranges.xlsx from ./data/
aed_ref_ranges_df = pd.read_excel("../../Data/aed_ref_ranges.xlsx")
# Lowercase Drug column
aed_ref_ranges_df["Drug"] = aed_ref_ranges_df["Drug"].str.lower()
# show unique units
print(aed_ref_ranges_df["Unit"].unique())
# mg/L and ug/mL are the same
# If Unit is ng/mL, convert to ug/mL
aed_ref_ranges_df.loc[aed_ref_ranges_df["Unit"] == "ng/mL", "Min"] = (
    aed_ref_ranges_df["Min"] / 1000
)
aed_ref_ranges_df.loc[aed_ref_ranges_df["Unit"] == "ng/mL", "Max"] = (
    aed_ref_ranges_df["Max"] / 1000
)
# Add a column that takes the average of Min and Max
aed_ref_ranges_df["Avg"] = (aed_ref_ranges_df["Min"] + aed_ref_ranges_df["Max"]) / 2
aed_ref_ranges_df

['mg/L' 'ug/mL' 'ng/mL']


,Drug,Min,Max,Unit,Avg
0,levetiracetam,12.00,46.00,mg/L,29.000
1,carbamazepine,4.00,10.00,mg/L,7.000
2,oxcarbazepine,3.00,35.00,ug/mL,19.000
3,clobazam,0.03,0.30,ng/mL,0.165
4,n-desmethylclobazam,0.30,3.00,ng/mL,1.650
5,topiramate,5.00,20.00,mg/L,12.500
6,valproic acid,50.00,125.00,ug/mL,87.500
7,lacosamide,1.00,10.00,ug/mL,5.500
8,felbamate,30.00,60.00,ug/mL,45.000
9,lamotrigine,2.50,15.00,mg/L,8.750


In [9]:
# frequency_bands = ["broadband", "60_100", "100_125"]
frequency_bands = ["broadband"]

In [10]:
for i, row in patients_df.iterrows():
    # Get patient id and weight
    patient_hup_id, patient_weight = row.hup_id, row.weight_kg
    patient_idx = patient_hup_id_to_index[patient_hup_id]
    print(f"Processing HUP {patient_hup_id}")

    # Find the ieeg_offset_1 value for patient_hup_id in ieeg_offset_df and convert it into float
    ieeg_offset_seconds = float(
        ieeg_offset_df.loc[
            ieeg_offset_df["hup_id"] == patient_hup_id, "ieeg_offset_1"
        ].values[0]
    )
    print(f"ieeg_offset_seconds: {ieeg_offset_seconds}")
    ieeg_offset_minutes = ieeg_offset_seconds / 60

    ##############################################
    # MEDICATIONS
    ##############################################
    # Load HUP_{patient_hup_id}.npy from ../../Data/medications
    aed_np_file = np.load(
        f"../../Data/medications/HUP_{patient_hup_id}.npy", allow_pickle=True
    )

    all_dose_curves_plot = aed_np_file[0]
    all_tHr_plot = aed_np_file[1]
    all_med_names_plot = aed_np_file[2]

    # Construct the time axis
    emu_start_time_hrs = min([all_tHr_plot[i][0] for i in range(len(all_tHr_plot))])
    emu_end_time_hrs = all_tHr_plot[0][-1]
    max_length = max([len(all_tHr_plot[i]) for i in range(len(all_tHr_plot))])
    time_axis = np.linspace(emu_start_time_hrs, emu_end_time_hrs, max_length)

    first_emu_hr = time_axis[0]

    # Create a dataframe that will hold the dose curves for all patients
    hourly_patient_features_df = pd.DataFrame(columns=["emu_time"])
    hourly_patient_features_df["emu_time"] = time_axis

    for potential_med_name in all_med_names:
        hourly_patient_features_df[f"med_{potential_med_name}_raw"] = np.zeros(
            len(time_axis)
        )

    sum_array = []

    ##############################################
    # MEDICATIONS Normalize to 1
    ##############################################
    for med_idx, med_name in enumerate(all_med_names_plot):
        dose_times = all_tHr_plot[med_idx].flatten()
        dose = all_dose_curves_plot[med_idx].flatten()

        interp_func = interpolate.interp1d(
            dose_times, dose, bounds_error=False, fill_value=0
        )
        dose_interp = interp_func(time_axis)

        if med_name != "lorazepam":
            sum_array.append(dose_interp)

        hourly_patient_features_df[f"med_{med_name}_raw"] = dose_interp

    cumulative_dose_curve = np.sum(sum_array, axis=0)
    cumulative_dose_curve = cumulative_dose_curve / np.max(cumulative_dose_curve)

    assert len(cumulative_dose_curve) == len(
        time_axis
    ), "cumulative_dose_curve and time_axis should have the same length"

    hourly_patient_features_df["med_sum_no_lorazepam_raw"] = cumulative_dose_curve

    ##############################################
    # MEDICATIONS Normalize with DDD
    ##############################################
    for med_idx, med_name in enumerate(all_med_names_plot):
        dose_times = all_tHr_plot[med_idx].flatten()

        # Find Avg for medication med_name in aed_ref_ranges_df
        if med_name != "lorazepam":
            ref_range = float(
                aed_ref_ranges_df.loc[
                    aed_ref_ranges_df["Drug"] == med_name, "Avg"
                ].values[0]
            )
        else:
            ref_range = 1

        dose = all_dose_curves_plot[med_idx].flatten()
        dose = dose / ref_range

        interp_func = interpolate.interp1d(
            dose_times, dose, bounds_error=False, fill_value=0
        )
        dose_interp = interp_func(time_axis)

        if med_name != "lorazepam":
            sum_array.append(dose_interp)

        hourly_patient_features_df[f"med_{med_name}_raw"] = dose_interp

    cumulative_dose_curve = np.sum(sum_array, axis=0)

    assert len(cumulative_dose_curve) == len(
        time_axis
    ), "cumulative_dose_curve and time_axis should have the same length"

    hourly_patient_features_df["med_sum_no_lorazepam_ddd"] = cumulative_dose_curve

    ##############################################
    # Group by 2 minutes and compute mean
    ##############################################
    hourly_patient_features_df["emu_minute"] = (
        (hourly_patient_features_df["emu_time"] * 60).astype(int) // 2 * 2
    )
    hourly_patient_features_df = hourly_patient_features_df.groupby("emu_minute").mean()
    hourly_patient_features_df = hourly_patient_features_df.reset_index()
    hourly_patient_features_df = hourly_patient_features_df.drop(columns=["emu_time"])

    ##############################################
    # SEIZURE COUNT
    ##############################################
    seizure_times_sec = np.load(
        f"../../Data/seizures/source_mat/HUP_{patient_hup_id}.npy"
    )
    seizure_times_sec = seizure_times_sec + ieeg_offset_seconds

    # Convert seizure times from seconds to minutes
    seizure_times_min = seizure_times_sec / 60

    hourly_patient_features_df["had_seizure"] = np.zeros(
        len(hourly_patient_features_df), dtype=int
    )

    for sz_min in seizure_times_min[:, 0]:
        hourly_patient_features_df.loc[
            hourly_patient_features_df["emu_minute"] == int(sz_min) // 2 * 2,
            "had_seizure",
        ] += 1

    ##############################################
    # Time since last seizure
    ##############################################
    # Initialize the list and timer
    time_since_last_seizure = []
    timer = None

    # Loop through the dataframe and calculate the time since the last seizure
    for had_seizure in hourly_patient_features_df["had_seizure"]:
        if had_seizure == 1:
            timer = 0
        elif timer is not None:  # if there has been a seizure before
            timer += 2
        else:
            timer = None
        time_since_last_seizure.append(timer)

    # Add the list as a new column
    hourly_patient_features_df["time_since_last_seizure"] = time_since_last_seizure

    ##########################################
    # SYNCHRONY
    ##########################################

    # Determine the starting index for the synchrony data
    start_index = None
    for i, emu_min in enumerate(hourly_patient_features_df["emu_minute"]):
        if i < len(hourly_patient_features_df["emu_minute"]) - 1:
            next_emu_min = hourly_patient_features_df["emu_minute"].iloc[i + 1]
        else:
            next_emu_min = emu_min + 2

        if emu_min <= ieeg_offset_minutes < next_emu_min:
            start_index = i
            break

    if start_index is None:
        print("start_index is actually 0...")
        start_index = 0

    for frequency_band in frequency_bands:
        synchrony_np = np.load(
            f"../../Data/synchrony/all/broadband_new_after_fill/HUP_{patient_hup_id}.npy"
        )

        # Initialize the synchrony column with NaNs
        hourly_patient_features_df[f"synchrony_{frequency_band}"] = np.nan

        # Insert synchrony values starting from the appropriate index
        end_index = min(
            start_index + len(synchrony_np), len(hourly_patient_features_df)
        )
        hourly_patient_features_df.iloc[
            start_index:end_index,
            hourly_patient_features_df.columns.get_loc(f"synchrony_{frequency_band}"),
        ] = synchrony_np[: end_index - start_index]

    ##########################################
    # AD Ratio
    ##########################################
    mat_file = scipy.io.loadmat(
        f"../../../erinconr/projects/fc_toolbox/results/analysis/intermediate/HUP{patient_hup_id}.mat"
    )
    mat_file = mat_file["summ"][0][0]
    ad_ratio = mat_file[17]
    num_channels = mat_file[6].shape[0]
    assert ad_ratio.shape[0] == num_channels

    ad_ratio = np.nanmean(ad_ratio, axis=0)
    ad_ratio = (ad_ratio - np.nanmedian(ad_ratio)) / iqr(ad_ratio, nan_policy="omit")
    assert np.nansum(ad_ratio) != 0

    # Reshape ad_ratio to match the granularity of the dataframe
    reshaped_ad_ratio = np.repeat(ad_ratio, 5)

    # Initialize the ad_ratio column with NaNs
    hourly_patient_features_df["ad_ratio"] = np.nan

    # Insert reshaped_ad_ratio values starting from the appropriate index
    end_index_ad_ratio = min(
        start_index + len(reshaped_ad_ratio), len(hourly_patient_features_df)
    )
    hourly_patient_features_df.iloc[
        start_index:end_index_ad_ratio,
        hourly_patient_features_df.columns.get_loc("ad_ratio"),
    ] = reshaped_ad_ratio[: end_index_ad_ratio - start_index]

    ##########################################
    # EEG time
    ##########################################
    # Create the eeg_time column with NaN values
    hourly_patient_features_df["eeg_time"] = np.nan

    # Define the eeg_time values starting from start_index
    eeg_time_values = np.arange(
        0, (end_index_ad_ratio - start_index) * 2, 2
    )  # incrementing by 2 since the time is grouped by 2 minutes
    hourly_patient_features_df.iloc[
        start_index:end_index_ad_ratio,
        hourly_patient_features_df.columns.get_loc("eeg_time"),
    ] = eeg_time_values

    ##############################################
    # SAVE TO CSV
    ##############################################

    hourly_patient_features_df.to_csv(
        f"../../Data/giant_tables_filled_only/HUP_{patient_hup_id}.csv", index=False
    )

Processing HUP 217
ieeg_offset_seconds: 143982.0


/tmp/ipykernel_36591/3367389285.py:209: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 210
ieeg_offset_seconds: 53647.0
start_index is actually 0...


/tmp/ipykernel_36591/3367389285.py:209: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)
/tmp/ipykernel_36591/3367389285.py:238: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  hourly_patient_features_df.iloc[


Processing HUP 206
ieeg_offset_seconds: 68154.0


...

/tmp/ipykernel_36591/3367389285.py:209: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 143
ieeg_offset_seconds: 130560.0


/tmp/ipykernel_36591/3367389285.py:209: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 139
ieeg_offset_seconds: 133613.0


/tmp/ipykernel_36591/3367389285.py:209: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)
